In [ ]:
import pandas as pd
import os


def combine_csv_files(folder_path, output_file_name):
    """
    Combines data from multiple CSV files in a specified folder into one.

    Args:
        folder_path (str): The path to the folder containing the CSV files.
        output_file_name (str): The name of the new CSV file to save the combined data to.
    """
    all_data = (
        pd.DataFrame()
    )  # Initialize an empty DataFrame to store all combined data

    # Iterate through all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {filename}")
            try:
                # Read the CSV file
                # You might need to specify encoding if you encounter errors (e.g., encoding='utf-8')
                df = pd.read_csv(file_path)
                all_data = pd.concat([all_data, df], ignore_index=True)
            except Exception as e:
                print(f"Could not read '{filename}': {e}")

    if not all_data.empty:
        # Save the combined data to a new CSV file
        # You might want to specify encoding here too (e.g., encoding='utf-8')
        output_file_path = os.path.join(os.path.dirname(folder_path), output_file_name)
        all_data.to_csv(output_file_path, index=False)
        print(f"\nSuccessfully combined data into: {output_file_path}")
    else:
        print("\nNo data was combined. Please check folder path and file types (.csv).")


# --- Configuration ---
# IMPORTANT: Replace 'path/to/your/csv_files' with the actual path to your folder
csv_files_folder = r"C:\Users\oystu\Desktop\Waterloo\2. Spring 2025\ECE 720 - ML for Chip Design\Project\Project\Dataset"  # Example: r"C:\Users\YourName\Documents\my_csv_files" or "/Users/YourName/Documents/my_csv_files"
output_csv_name = "Combined_ProcessedData.csv"

# Run the function
combine_csv_files(csv_files_folder, output_csv_name)

In [ ]:
import os
import pandas as pd


def combine_all_rates_for_each_region(folder_path, output_filename):
    """
    Finds sets of 7G, 14G, 28G, and 56G files, merges their unique
    columns for each set, and then combines all sets into a single output file.
    """
    print(f"Searching for base files (7G) in: {folder_path}")

    try:
        all_filenames = os.listdir(folder_path)
        base_filenames_7g = [f for f in all_filenames if f.endswith("_7G_input.0.csv")]
        base_files_7g = [os.path.join(folder_path, f) for f in base_filenames_7g]
    except FileNotFoundError:
        print(f"Error: The specified folder was not found -> {folder_path}")
        return

    if not base_files_7g:
        print(
            "No '..._7G_input.0' files found. Please check the folder path and filenames."
        )
        return

    print(f"Found {len(base_files_7g)} sets to process.")

    list_of_all_processed_dfs = []

    for file_7g_path in base_files_7g:
        set_name = os.path.basename(file_7g_path).split("_region_")[0]
        print(f"\nProcessing set: {set_name}")

        try:
            print(f"  -> Reading base file: {os.path.basename(file_7g_path)}")
            # MODIFICATION: Removed sep=r'\s+' to read as a standard CSV
            combined_df = pd.read_csv(file_7g_path)

            rates_to_merge = ["14G", "28G", "56G"]

            for rate in rates_to_merge:
                next_file_path = file_7g_path.replace("_7G_input.0", f"_{rate}_input.0")
                if not os.path.exists(next_file_path):
                    print(
                        f"  -> Warning: {os.path.basename(next_file_path)} not found. Skipping."
                    )
                    continue

                print(f"  -> Merging columns from: {os.path.basename(next_file_path)}")
                # MODIFICATION: Removed sep=r'\s+' to read as a standard CSV
                next_df = pd.read_csv(next_file_path)

                if len(next_df) != len(combined_df):
                    print(
                        "  -> ❌ Error: Row count mismatch between files. Aborting this set."
                    )
                    combined_df = None
                    break

                unique_cols = list(set(next_df.columns) - set(combined_df.columns))
                if unique_cols:
                    print(f"    -> Adding {len(unique_cols)} new unique columns.")
                    combined_df = pd.concat([combined_df, next_df[unique_cols]], axis=1)
                else:
                    print("    -> No new unique columns found.")

            if combined_df is not None:
                list_of_all_processed_dfs.append(combined_df)
                print(
                    f"  -> ✅ Set '{set_name}' processed and added to the final combination."
                )

        except Exception as e:
            print(
                f"  -> ❌ An unexpected error occurred while processing set '{set_name}': {e}"
            )

    if not list_of_all_processed_dfs:
        print("\nNo data was successfully processed to be combined.")
        return

    print("\nCombining data from all processed sets...")
    final_master_df = pd.concat(list_of_all_processed_dfs, ignore_index=True)

    # Reorder columns to have parameters first, which are not present in this flow, but good practice
    param_order = ["fW", "current", "ind", "Rd", "Cs", "Rs"]
    existing_params = [p for p in param_order if p in final_master_df.columns]
    other_cols = [c for c in final_master_df.columns if c not in existing_params]
    final_master_df = final_master_df[existing_params + other_cols]

    output_path = os.path.join(folder_path, output_filename)
    final_master_df.to_csv(output_path, index=False)
    print(f"\n✅✅✅ All data successfully combined into: {output_path}")


# --- Configuration ---
data_folder = r"C:\Users\oystu\Desktop\Waterloo\2. Spring 2025\ECE 720 - ML for Chip Design\Project\ECE720T32-Using-ML-for-CTLE-Circuit-Generation\dv3"
final_output_name = "DataCombined.csv"

# --- Run the Script ---
combine_all_rates_for_each_region(data_folder, final_output_name)
print("\nProcess finished.")

In [ ]:
import pandas as pd
import os


def combine_csv_files(folder_path, output_filename):
    """
    Finds all CSV files starting with 'DataCombined' using os.listdir(),
    and concatenates them into a single output file.
    """
    try:
        # --- MODIFICATION START ---
        # Use os.listdir() to get all filenames in the folder
        all_filenames_in_dir = os.listdir(folder_path)

        # Filter the list to find files that match our criteria
        matching_filenames = [
            f
            for f in all_filenames_in_dir
            if f.startswith("DataCombined") and f.endswith(".csv")
        ]

        # Create the full path for each matching file
        file_list = [os.path.join(folder_path, f) for f in matching_filenames]
        # --- MODIFICATION END ---

    except FileNotFoundError:
        print(f"⚠️ Error: The specified folder was not found -> {folder_path}")
        return

    if not file_list:
        print(
            f"⚠️ No files starting with 'DataCombined' and ending with '.csv' were found in the folder."
        )
        return

    print(f"Found {len(file_list)} files to combine.")

    list_of_dataframes = []

    for file_path in file_list:
        try:
            print(f"Reading: {os.path.basename(file_path)}")
            df = pd.read_csv(file_path)
            list_of_dataframes.append(df)
        except Exception as e:
            print(f"  -> ❌ Error reading file {os.path.basename(file_path)}: {e}")

    if not list_of_dataframes:
        print("No data could be read. Aborting combination.")
        return

    print("\nCombining all data...")
    final_combined_df = pd.concat(list_of_dataframes, ignore_index=True)

    output_path = os.path.join(folder_path, output_filename)
    final_combined_df.to_csv(output_path, index=False)

    print(f"\n✅ Success! All files have been combined into '{output_filename}'.")
    print(f"Total rows in new file: {len(final_combined_df)}")


# --- Configuration ---
# Please confirm this is the correct folder containing your 'DataCombined' files.
data_folder = r"C:\Users\oystu\Desktop\Waterloo\2. Spring 2025\ECE 720 - ML for Chip Design\Project\ECE720T32-Using-ML-for-CTLE-Circuit-Generation\combinedData"
output_file = "DataV2.csv"

# --- Run the Script ---
combine_csv_files(data_folder, output_file)